This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 150.37102595, 325.49788774, ...,  33.52619036,
        19.41019935,  47.20790294]), array([230.58254373, 165.09559185, 264.22206753, ...,  21.29948064,
        18.8990421 ,  67.99229862]), OrderedDict([('R2', 0.7684167866863664), ('root_mean_squared_error', 47.25262434474679), ('mean_absolute_error', 32.097495763419516), ('rmse_over_stdev', 0.48503097781797927)]), None)
test_quad = (array([202.0924573 , 222.92732066, 229.79012828, 214.18575236,
       209.63441278, 407.14718606, 396.88333541, 143.14427   ,
       110.503751  ,  95.94694389,  95.34311114, 121.012896  ,
       127.30671062,  99.38751557,  97.75371527,  89.46412177,
       170.11252586, 144.36770295, 190.00207148, 170.82835436,
       187.70991454, 302.98586264, 114.36792625,  70.6664565 ,
        91.15693652,  74.58551681,  83.80925664, 228.9145309 ,
        70.2166732 ,  45.07319863,  97.86627057, 118.30376345,
        87.31329665,  98.51156618, 417.7519315 ,  99.06622089,
        66.0766101 ,  86.04966315,  97.04621619, 111.44229377,
        60.35669252, 132.8149808 ,  88.11493139,  86.43036922,
       182.74091205,  63.36273585,   0.        , 643.729766  ,
        69.43614177,  57.32200585,  69.28696139,  63.68903168,
        36.52821981,  12.9458568 ,   0.        ,  50.0993    ,
         0.        ,   0.        , 128.57645296,   0.        ,
        64.66274573,  53.57771377,   0.        ,  29.16320729,
        78.02361062, 130.86141575,   0.        , 140.13445715,
        79.33429315, 236.69985867, 127.82578903, 198.10666422,
       206.15823625, 288.48656282, 251.77174365, 377.5020086 ,
       119.77498325, 219.40213351,  59.63498938,  75.15595576,
         0.        ,  45.21596181,  34.47065573,  27.99263983,
         0.        ,   0.        ,   0.        ,   0.        ,
        27.03043917,   0.        ,   0.        ,  60.52191244,
        40.83543394,  43.99569352, 157.37436322,  69.26938771,
         0.        ,   0.        ,  49.36322998,   0.        ,
         0.        ,   0.        ,   0.        ,  14.81005156,
        60.68027662,  74.7468692 ,  25.722252  ,  39.83120675,
         0.        ,  36.17951005,  73.3952221 , 119.11416769,
       137.40871252, 150.99987636,   0.        ,  98.58075515,
       185.40421066, 170.3886173 ,   0.        ,  93.67170962,
         0.        ,   8.691789  ,  93.42580713, 127.37818921,
       158.42704307, 285.94816437,  69.47351553, 257.0759627 ,
       284.89818957,  48.46453063,  39.9617942 , 280.87646981,
       139.76313312,  42.1689745 , 211.4607162 ,  31.5202492 ,
       281.10244257, 126.7773445 , 175.56939781, 140.53855037,
       139.34508133, 367.6244316 , 211.47682381,  57.38221378,
        25.02448734,  29.18020703,  13.18080166,  28.05755061,
        13.89243768,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,  55.47285962,  72.01080223,
         0.        ,  52.52667425,  48.18377975,  48.37559255,
        58.54651811,  47.92046585,   0.        ,  69.81139111,
        91.38113959,  64.51928629,  69.47703167,  82.92662983,
       202.39359151,  46.33040675,  82.66465971,  54.66615752,
        50.70809969,  41.29569684,  34.25106067, 224.40216532,
       141.4711659 , 154.69746659,  93.87112816,  91.34499746,
         0.        ,  65.98496318,  28.58352225,  25.71300756,
        27.59599462,  51.75595534,  42.74924617,  51.95072185,
        47.24337609,  95.22684338,  71.18482569,  70.13978156,
        94.05845519,  14.00114963, 178.13772044,  32.611025  ,
        23.21180706,   0.        ,   5.20671817,  26.1974495 ,
         0.        ,  44.54672265, 191.11304901, 179.41288932,
       155.66304648, 250.25203832, 260.12973722, 286.02658903,
       183.82868108, 165.1182213 ,  88.32721153,  99.68134329,
        97.79037938, 187.87580019, 241.66495314, 196.07415857,
       114.40771587, 153.2465035 , 114.50715188, 130.34718776,
       189.95220813, 189.82462955, 114.4981617 , 149.94833269,
       189.88100424, 124.31412396, 150.67246806,  98.1125625 ,
       122.67765683, 140.40595738,  87.04599469, 319.11663064,
       237.99271494, 309.85662145, 199.00234553, 205.56403043,
        70.82208722, 110.20908761,  91.28475717,  83.54954986,
       166.21281277, 280.85429217, 100.11742673, 140.26518176,
       159.4014052 , 154.70761089, 253.01986447, 238.52227037,
       252.00105048, 298.52579257, 323.63415062, 318.43464993,
       164.00223776,  79.9859482 , 150.04202796,  79.32480591,
       223.62476104, 136.44308822, 151.8983111 ,  64.02417181,
       139.23135132,  59.52415623, 138.09122581,  61.57636317,
       126.81817022, 117.79227682, 193.51782743, 312.11562515,
       178.51986829, 299.11257829, 144.13504497, 191.40453615,
       200.76049289, 347.29590062, 151.15513036, 161.6376499 ,
       181.01990191, 135.42648917, 105.93237224,  98.80490655,
       123.3670698 ,  78.21525252, 141.35404325,  46.57820663,
       110.06584633, 109.91309651, 131.79861623,  94.52083822,
       106.91311671, 156.51808854,  96.47357254, 125.31262584,
        36.00668628, 117.38941205,  27.21450957,  29.13729713,
       124.27311697,  24.231722  ,  32.64732478,  73.45957459,
        30.53286251,  33.18389082, 131.67141422, 133.6354298 ,
        41.58359043,  51.27623451, 205.07521682, 234.93787541,
       149.54096509, 140.21882844,  95.20604545, 211.33526584,
       184.21597694, 160.45672687, 113.5272932 , 184.77929222,
       170.29275428, 182.10184863, 236.86191352, 163.79391125,
       146.5521539 , 111.38413582, 142.93338794, 181.38757478,
       192.12866913, 220.40870164,  54.91523627,  30.77112483,
       124.54211081,  83.56862417, 137.47965258, 105.72217961,
       109.48895599, 113.88620361, 102.77542034,  96.03603838,
       117.24705906, 104.24420754, 188.51471262, 184.59131561,
       291.09907332, 110.86471789, 216.12219457, 141.96565963,
       135.36931657, 121.69241919, 163.49712402, 244.71754232,
        82.87910056,  29.74770732,  42.13350699,  44.18158824,
        34.82219739, 197.61006658,  71.09076358,  22.37923032,
        87.12994833,  59.14294608, 112.94910648, 165.73307859,
        49.79361192,  98.84644072, 140.48883536, 161.96626836,
       132.01998148, 182.41715959, 179.36951362, 129.16762848,
        38.13605333, 123.68971028,   0.        ,  62.09475103,
        49.53238936,   0.        , 112.1749515 ,  71.88859602,
       124.75938063,  24.15276207,   4.75557533,  49.21928699,
        56.33881581,  59.2192098 , 100.47292181,  61.55780711,
        43.46433055,  21.35392387,  28.48129225,  37.37694705,
        14.2008055 ,  84.72896518, 120.02645236,  74.81814061,
        27.11965388,  38.65176325,  27.79921436,   0.        ,
         0.        ,  81.726855  , 221.76800698,  40.70021975,
        91.67074296,  69.473884  ,  91.10598761,  39.11088786,
        52.14150703,  46.95075044,  43.94549002,  84.26131862,
        86.51921261,  71.76198016,   0.        ,  31.52290816,
       142.32388436,  46.35809831,  30.59603878,  46.76679762,
        80.15612571,  35.37179875,   7.0009685 ,  77.68295   ,
        27.90730325,  38.754046  ,   0.        ,   0.        ,
         0.        ,  36.55454693,   0.        ,  23.809066  ,
         0.        ]), array([135.66349937, 204.81330652, 168.69624488, 168.74102132,
       151.35561536, 269.49797071, 261.41174811, 120.83832981,
       121.0146437 ,  94.52251002,  91.97274348, 141.69411913,
       123.72796365, 126.53731467,  97.9280815 ,  85.76788937,
       170.46745618, 226.98029234, 169.35211447, 167.02439598,
       159.95808432, 212.68710843, 156.76402782,  90.76320044,
        70.2348966 ,  92.16181891,  89.89659336, 215.93619205,
        37.26336158,  42.743955  , 129.02266146, 129.8851179 ,
        85.16412912,  74.98685678, 247.21820414,  80.32993541,
        93.00686658, 106.16623311, 106.71235838, 117.56184546,
        77.18744515,  75.71085802,  68.87214254, 100.98249102,
       180.61051282,  24.01936708, 123.72715251, 311.98386996,
        99.01399074,  96.97670639, 131.98552642, 202.45523472,
       148.89855849, 112.52652005, 110.71531598,  36.27190138,
        17.27500766,  94.35238216, 155.59845755,  18.77998446,
       103.28373698, 103.3103714 ,  26.46124525,  25.76623124,
        51.34403236,  30.84578891,  29.32646727, 159.46164865,
        53.82094699,  66.38636683, 123.70872187, 153.27105028,
       109.41889225, 281.91030227, 168.6285197 , 285.93206834,
       142.55681887, 201.73795285,  75.5960377 ,  84.94872689,
        -0.79399902,  84.70813413,  55.70577955,  54.04363336,
         7.47954728,  35.12866033,  -1.44826339,  17.81833563,
        26.18359774,  58.44761309,  18.41592662,  42.63184018,
        34.99938979,  28.56764465, 108.99027199,  84.06687978,
        25.20250966,  13.79390211,  80.08766473,  24.28171866,
         9.03327687,  35.05244686,  14.86415846,  28.09350722,
        44.18864143,  31.18247099,  22.98851494,  57.63252625,
        13.15783993,  21.22334072,  20.82866766,  63.55535817,
       117.83096127, 121.71958969,  35.23102819,  67.62487676,
       176.8762925 , 126.54957548,  23.5979318 , 120.54911708,
        39.32732437,  40.18091651, 182.83236161, 159.41974847,
       175.12756778, 193.67885177,  92.58843863, 296.97364379,
       310.89602314,  94.19337852,  56.41775775, 232.06358677,
        76.5003319 ,  22.08971169,  34.87303907,  30.70409841,
       192.84766521,  25.20591254,  53.12261328, 113.68542444,
       111.29193631, 204.05363243,  86.26086207, 182.19755468,
        58.37198059,  36.37093675,  20.18263074,  68.32022303,
        32.50890584,  -1.79437598,  13.5870183 ,   9.58486984,
         3.56995273,  12.96445483,  58.35418195,  65.45784496,
        -2.13771447,  92.19029306,  48.51068944,  58.98271089,
        65.63212307,  25.69853974,  20.18096336,  56.67052594,
        69.8788911 ,  71.30058105,  74.54591897,  79.41814854,
       161.0048814 ,  72.60869409,  63.60746434,  71.43632434,
        53.60200422,  14.36854261,  41.7878111 , 232.4608373 ,
       130.34367223, 155.48390286,  86.5470965 ,  95.57259175,
        36.55704227,  61.95471769,  33.74848174,  44.01743809,
        47.86354401,  47.17848242,  28.89028105,  11.80256249,
        82.25172422,  43.2867442 ,  42.92584472,  41.56941312,
        63.45973426,   2.57707379,  49.80279502,  35.63927177,
        31.30690105,  21.68146959,   7.84199293,  11.06914441,
         9.61442234,  43.53292469, 236.7049248 , 142.31382121,
       144.67008118, 307.73536579, 282.77226887, 266.94278065,
       137.80463866, 130.29419478, 163.48943351, 119.63332702,
       127.41064529, 162.0953149 , 297.48934639, 225.22618615,
        89.80584885,  92.75553513,  86.86349863,  94.12072484,
       208.73132814, 178.38363885,  87.4347852 ,  88.16232808,
       142.66932196, 151.7304053 , 162.49441154, 105.0305909 ,
        85.49317147,  95.92277   ,  79.65457438, 191.04202021,
       304.34913319, 289.12503862, 169.89725745, 209.28897211,
        81.55170524,  86.66436734,  95.42079072, 139.04046828,
       198.36433347, 197.39897278, 101.23620268, 124.04949806,
       162.35944826, 140.96630476, 228.44134315, 212.56400742,
       213.42108021, 274.96859039, 296.17385741, 189.04047942,
       135.40859285,  86.12633055, 147.24396139, 142.41887859,
       270.96030876, 170.58819846, 136.30779492,  83.42402156,
       129.21332682,  98.97126341, 132.52420047,  65.35414539,
       124.98391851, 139.5956234 , 221.72626948, 301.97812149,
       204.10882904, 237.50807587, 185.6685276 , 206.93407093,
       222.78770087, 300.47823282, 163.46061018, 144.04947496,
       124.33484937, 138.45149812, 134.04278125, 124.60744133,
       144.98149687, 124.59418618, 132.58433353, 108.25926703,
       141.91200749, 132.23917139, 125.75646233, 128.03652251,
       136.43783882, 138.05923928, 127.04302003, 146.50139512,
        58.5695245 ,  69.51045319,  48.80504193,  39.56755864,
        83.793058  , 100.16723456, 107.94856736, 114.2124182 ,
        72.36805348,  39.28792299, 136.29361576,  69.34301572,
        70.31087255,  39.21539414, 174.2387052 , 164.69657069,
       176.93294016, 240.17749783, 163.36930868, 120.34027098,
       176.66599721, 162.85868557, 112.60914895, 167.00782249,
       246.12592742, 172.06992309, 158.95584299, 179.04373308,
       171.29439951, 115.53426091, 133.92282757, 241.2059202 ,
       168.67867358, 156.29835303, 104.49687151,  49.43992434,
       226.52569161, 124.86717323, 161.7931827 , 117.30001507,
       133.16053133, 130.38963025, 109.18731352, 107.27660949,
       120.4395898 , 114.90318311, 213.53164265, 118.59561559,
       205.1651076 , 104.30051479, 195.78280653, 118.14288468,
       108.0682663 , 110.31581072, 148.94365104, 199.90164558,
       101.0356055 ,  67.63853605,  70.60227451,  42.36768114,
        37.87131109,  91.73695928,  78.1853434 ,  46.95316369,
        32.88994168,  75.10243361,  50.33931455,  86.08010345,
        50.38462549, 107.5321599 , 108.11305654, 160.78145748,
        48.67715547,  81.37990046, 131.12943381, 112.25331312,
        57.01103964, 107.26418865,   8.28861668,  61.47721193,
        55.08074062,   7.8477628 ,  45.2073685 ,  49.50515106,
        81.84142254,  50.06322286,  23.44361273,  45.22943803,
        98.55028132, 120.5267174 ,  87.86395445,  72.02305453,
        27.2474601 ,   2.1297083 ,   8.32050524,  64.44426943,
        18.71229243,  61.43351724,  70.06081527,  69.08186213,
        28.71999239,  30.60815789,  32.33195464,  35.31371884,
        47.32987901,  75.87290212, 151.64891689,  59.14808525,
        38.2290169 ,  33.76176224,  52.86070105,  62.01857386,
        38.1402095 ,  30.79930656,  22.09035716, 147.88549047,
       159.3232871 ,  77.02942102,  72.6746278 ,  67.50342811,
        88.94535146,  32.07095395,  19.42640921,  34.09482648,
        77.54190377,  41.5043661 ,  57.38805828,  48.84157456,
        23.03615453,  25.06743968,  21.20911398,   9.96222391,
        23.13946778,  22.3912519 ,  74.39672409,  53.19776538,
         8.5577464 ]), OrderedDict([('R2', 0.7037145944884045), ('root_mean_squared_error', 47.44265355402022), ('mean_absolute_error', 32.61733567908851), ('rmse_over_stdev', 0.4869815584357966)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/RepeatedKFold/split_4'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
